In [ ]:
!pip install msgpack

## Client Side

#### IMPORT BLOCK

In [ ]:
import msgpack
import hmac
import hashlib
import json
import socket
import hashlib
from os.path import exists

#### PARAMETERS BLOCK

In [ ]:
# PARAMETERS
max_nb_attempts = 5 # stop after max_nb_attempts unanswered auth requests
reception_timeout = 2 # wait up to reception_timeout sec for the response, if no answers are received in that time resend it by changing ctr
# Service for which authentication is asked 
server_ip = '10.0.0.7'
server_port = 22

#### FUNCTIONS BLOCK

In [ ]:
# Generate and returns an auth request ready to be sent
def make_auth_request(server_ip, server_port, ctr, transport_layer, client_ip, shared_key):
    # json request
    request_data_dict = {
      'server': server_ip,
      'dport': server_port,
      'ctr': ctr,
      'tcp_udp' : transport_layer,
      'my_ip': client_ip, 
      'hmac': 0
    }

    # Serialize request_data_dict
    request_data_raw = msgpack.packb(request_data_dict)

    # Generate a message authentication code of request_data_raw based on 
    # the shared_key and secure hashing algorithm SHA256 using hmac module
    hmac1 = hmac.new(key=shared_key.digest(), digestmod=hashlib.sha256)
    hmac1.update(request_data_raw)
    message_digest = hmac1.digest()
    
    # Insert the computed HMAC in hmac value of the request dictionary
    request_data_dict['hmac'] = message_digest
    
    # Serialize request_data_dict with the updated HMAC
    auth_request = msgpack.packb(request_data_dict)
    
    return auth_request

# Given the an auth request/reply message returns True if it is authenticated
def is_msg_authenticated(rcv_message, hash_key):
    rcv_hmac = rcv_message['hmac']

    # Compute the HMAC of the modified authentication request (hmac=0)
    rcv_message['hmac'] = 0
    modified_json = msgpack.packb(rcv_message)
    computed_hmac = hmac.new(key=hash_key.digest(), digestmod=hashlib.sha256)
    computed_hmac.update(modified_json)
    message_digest = computed_hmac.digest()

    # Compare the computed HMAC with the received one
    return hmac.compare_digest(rcv_hmac, message_digest)

# Create json configuration file (conf.json) containing initial values of controller_ip, auth_port, shared_key, next_ctr
def create_conf_file():
    conf_dict = {
        "controller_ip" : "10.0.0.1",
        "auth_port" : 50000,
        "master_key" : "abracadabra",
        "next_ctr" : 1
    }

    with open("conf.json", "w") as config_file:
        json.dump(conf_dict, config_file)

#### MAIN BLOCK

In [ ]:
if not exists('conf.json'):
    print('Configuration file not found, creating a new one...')
    create_conf_file()
    
# Read conf.json to retrieve parameters to perform the authentication request
with open('conf.json', 'r') as config_file:
    config_json = json.load(config_file)
    controller_ip = config_json["controller_ip"]
    auth_port = config_json["auth_port"]
    master_key = config_json["master_key"]
    ctr = config_json["next_ctr"]

# Get the two symmetric keys from the master key for HMAC generation
my_shared_key = hashlib.sha256(str.encode(str(master_key+'0')))
controller_shared_key = hashlib.sha256(str.encode(str(master_key+'1')))
print("Obtained the following shared keys:\n\tclient {}\n\tcontroller {}".format(my_shared_key.hexdigest(), controller_shared_key.hexdigest()))

# Retrieve my ip -> required later for auth request
local_ip = socket.gethostbyname(socket.gethostname())

# --- Perform the authentication request ---
has_srv_replied = False
nb_attempts = 0
authorised_by_controller = False

while (not authorised_by_controller) and (nb_attempts < max_nb_attempts):
    # Generate auth request
    auth_request = make_auth_request(server_ip, server_port, ctr, 'tcp', local_ip, my_shared_key)
    # Use socket to send the auth request and retrieve the auth reply from controller
    sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    sock.settimeout(reception_timeout)
    sock.sendto(auth_request, (controller_ip, auth_port))
    nb_attempts += 1
    print("Attempt n.{}: Authentication request sent with counter {}".format(nb_attempts, ctr))
    try:
        rcv_message, rcv_address = sock.recvfrom(1024)
        rcv_ip = rcv_address[0]
        # Accept responses only from controller IP
        if rcv_ip == controller_ip:
            # check authentication
            if is_msg_authenticated(rcv_message, controller_shared_key):
                has_srv_replied = True
                # check if controller granted the authentication
                unpacked_auth_reply = msgpack.unpackb(rcv_message)
                print('\tController replied with {}'.format(unpacked_auth_reply['code']))
                if unpacked_auth_reply['code'] == "202 Authentication Accepted":
                    authorised_by_controller = True
                    # Update ctr
                    ctr = unpacked_auth_reply['expected_ctr']
                elif unpacked_auth_reply['code'] == "449 Retry With":
                    print('\tUpdating counter to {} and increasing max_nb_attempts value'.format(unpacked_auth_reply['expected_ctr']))
                    ctr = unpacked_auth_reply['expected_ctr']
                    max_nb_attempts = max_nb_attempts + nb_attempts                    
            else:
                print('Received a non authenticated message from controller IP')
                ctr += 1            
        else:
            print("\tReceived a packet but not from controller ({})".format(rcv_ip))
    except socket.timeout:
        print("\tReply not received from controller")
        ctr += 1 # Should we increase the counter here?
        continue

# Once the authentication phase has been completed successfully, close the socket
sock.close()

# In case of no answer from controller or no authorization -> stop here
if not has_srv_replied:
    print('*** Unable to reach the controller after {} attempts. Try later ***'.format(max_nb_attempts))
    exit()
elif not authorised_by_controller:
    print('*** Not authorized by controller. Check the shared key ***'.format(max_nb_attempts))
    exit()
    
# --- Update next_ctr value in config.json ---
with open('conf.json', 'r') as config_file:
    config_json = json.load(config_file)

config_json["next_ctr"] = ctr

with open('conf.json', 'w') as config_file:
    json.dump(config_json, config_file)
    
# OPEN SOCKET WITH SERVER


## Controller Side

####  IMPORT BLOCK

In [ ]:
import msgpack
import hmac
import hashlib
import json
import socket
import hashlib
from os.path import exists

####  PARAMETER BLOCK

In [ ]:
# Define the margin for which the ctr is accepted
ctr_margin = 5

#### FUNCTIONS BLOCK

In [ ]:
# Create json configuration file (conf.json) containing initial values of controller_ip, auth_port, shared_key, next_ctr
def create_conf_file():
    conf_dict = {
        "controller_ip" : "10.0.0.1",
        "auth_port" : 50000,
        "master_key" : "abracadabra",
        "next_ctr" : 1
    }

    with open("conf.json", "w") as config_file:
        json.dump(conf_dict, config_file)
        
        
# Given the unpacked authentication message,
# returns True if it is correct (it is a dict and contains at least the required fields),
# False otherwise
def is_req_auth_correct(unpacked_auth_msg):
    correct = False
    
    expected_fields = set(['server', 'dport', 'ctr', 'tcp_udp', 
                           'my_ip', 'hmac'])
    
    # check if it is a dictionary
    if type(unpacked_json) is dict:
        # Check that it contains all the expected fields
        if expected_fields.issubset(unpacked_auth_msg.keys()):
            correct = True
    return correct

# Given the an auth request/reply message returns True if it is authenticated
def is_msg_authenticated(rcv_message, hash_key):
    rcv_hmac = rcv_message['hmac']

    # Compute the HMAC of the modified authentication request (hmac=0)
    rcv_message['hmac'] = 0
    modified_json = msgpack.packb(rcv_message)
    computed_hmac = hmac.new(key=hash_key.digest(), digestmod=hashlib.sha256)
    computed_hmac.update(modified_json)
    message_digest = computed_hmac.digest()

    # Compare the computed HMAC with the received one
    return hmac.compare_digest(rcv_hmac, message_digest)

# Generate and returns an auth reply ready to be sent
def make_auth_reply(code, expected_ctr, shared_key):
    # json request
    reply_data_dict = {
      'code': code,
      'expected_ctr': expected_ctr,
      'hmac': 0
    }

    # Serialize request_data_dict
    reply_data_raw = msgpack.packb(reply_data_dict)

    # Generate a message authentication code of request_data_raw based on 
    # the shared_key and secure hashing algorithm SHA256 using hmac module
    hmac1 = hmac.new(key=shared_key.digest(), digestmod=hashlib.sha256)
    hmac1.update(reply_data_raw)
    message_digest = hmac1.digest()
    
    # Insert the computed HMAC in hmac value of the request dictionary
    reply_data_dict['hmac'] = message_digest
    
    # Serialize request_data_dict with the updated HMAC
    auth_reply = msgpack.packb(reply_data_dict)
    
    return auth_reply

#### MAIN BLOCK

In [ ]:
authorised = False

if not exists('conf.json'):
    print('Configuration file not found, creating a new one...')
    create_conf_file()
    
# Get master key and counter values from conf.json
with open('conf.json', 'r') as config_file:
    config_json = json.load(config_file)
    master_key = config_json["master_key"]
    expected_ctr = config_json["next_ctr"]

# Get shared keys from master key
client_shared_key = hashlib.sha256(str.encode(str(master_key+'0')))
controller_shared_key = hashlib.sha256(str.encode(str(master_key+'1')))
print("Obtained the following shared keys:\n\tclient {}\n\tcontroller {}".format(client_shared_key.hexdigest(), controller_shared_key.hexdigest()))

# ----- Analyze the received packet -----
# Unpack the received authentication message 
unpacked_auth_request = msgpack.unpackb(auth_request)

# Check its correctness (it is a json with all the required fields)
correct = is_req_auth_correct(unpacked_auth_request)

rcv_ctr = unpacked_auth_request['ctr']

if correct:
    authenticated = is_msg_authenticated(unpacked_auth_request, client_shared_key)
    if authenticated:
        print('Authenticated message:')
        print("\t", unpacked_auth_request)
        # Check if ctr is valid
        if rcv_ctr in range(expected_ctr, expected_ctr + ctr_margin):
            code = '202 Authentication Accepted'
            expected_ctr = rcv_ctr + 1
            authorised = True
        else: # authenticated but not valid ctr -> not accepted
            code = '449 Retry With'
    else: # message not authenticated
        code = '401 Unauthorized'
else: # received message not correct 
    code = '400 Bad Request'
    
# ----- Send the auth reply -----
# Generate auth reply
auth_reply = make_auth_reply(code, expected_ctr, controller_shared_key)
print(msgpack.unpackb(auth_reply))
# send it

# ----- Installing rules -----
#if authorised:
    # install the following rules
    # - route from client to server (priority 10 with a few seconds hard timer)
    # - match the reverse flow to controller to create a bidirectional routing (priority 20 with a few minutes idle timer)
